In [4]:
import nest_asyncio
nest_asyncio.apply()

from pathlib import Path
from lurawi.lurawi_agent import LurawiAgent

import uuid
from chromadb import PersistentClient
from chromadb.config import Settings
from chromadb.utils import embedding_functions

import chromadb.utils.embedding_functions as embedding_functions

from tqdm import tqdm


In [ ]:
def rag_encode_markdowns(source_dir: str, collection_dir: str):
    client = PersistentClient(path=collection_dir, settings=Settings(anonymized_telemetry=False))

    embedding_fn = embedding_functions.OpenAIEmbeddingFunction(
                    api_key="feind",
                    api_base="http://maxbackpack:8080/v1",
                    model_name="something"
                )

    collection = client.create_collection("summary_collection", embedding_function=embedding_fn)


    summary_agent = LurawiAgent(name="summary_agent", behaviour="summary_agent",
                                workspace="/home/xun/dev/lurawi")

    directory = Path(source_dir)
    if not directory.exists():
        raise FileNotFoundError(f"Directory not found: {source_dir}")
 
    md_files = directory.rglob("*.md")

    for md_file in tqdm(md_files):
        #print(f"processinng {md_file.name}...")

        summary = summary_agent.run_agent(summary_prompt="你是一个资深大厨师和美食家。用简洁的语言，从主要配料，制作难度，需要多少时间来准备，什么时令适合做，适合怎样体质的人食用，有什么益处 这几方面来评判总结以下菜谱。不要用表情包！",
                                text_file=str(md_file.resolve()))

        chunk_id = str(uuid.uuid4())
        collection.add(
                    documents=[summary],
                    ids=[f"doc_{chunk_id}"],
                    metadatas=[{"source": str(md_file.name), "chuck_id": chunk_id}]
                )


In [ ]:
rag_encode_markdowns(source_dir="/home/xun/dev/HowToCook/dishes")